In [1]:
# dataset retrieved from https://archive.ics.uci.edu/dataset/352/online+retail
import pandas as pd
import numpy as np

In [20]:
file_path = "data/Online_Retail.csv"

df = pd.read_csv(file_path,
                 header= 0,
                #  nrows= 5,
                 parse_dates= ["InvoiceDate"]
                 )

print(df.info())

# print(df.iloc[141])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


## Sanity check for null values

In [32]:
print("Number of NA values in each column:")
print(df.isna().sum())

total_na = df.isna().sum().sum()
print(f"Total number of NA values in the DataFrame: {total_na}")

# drop rows without descriptions
df = df.dropna(subset=['Description']) 

Number of NA values in each column:
InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64
Total number of NA values in the DataFrame: 133626


- 1454 rows without description (dropped)
- 135080 rows without CustomerID

In [36]:
nonpositive_count = (df["Quantity"] <= 0).sum()
print(f"There are {nonpositive_count} nonpositive values") # 9762 items with "Quantity" of 0 or less

df = df[df["Quantity"] > 0] # Drop nonpositive rows
df.reset_index(drop=True, inplace=True) # account for index of dropped rows
print(f"{nonpositive_count} nonpositive values dropped")

There are 0 nonpositive values
0 nonpositive values dropped
